In [1]:
# !pip3 install requests
# !pip3 install python-dotenv
# !pip install gspread
# !pip install oauth2client
# !pip install google-auth
# !pip install google-auth-oauthlib
# !pip install google-auth-httplib2

In [2]:
import os
import requests
import gspread
import datetime
from oauth2client.service_account import ServiceAccountCredentials
from dotenv import load_dotenv

In [3]:
load_dotenv('.env')

True

In [4]:
# Replace with your Canvas API token
access_token = os.getenv('CANVAS_ACCESS_TOKEN')
base_url = os.getenv('CANVAS_BASE_URL')

# Headers for authentication
headers = {
    'Authorization': f'Bearer {access_token}'
}

# Parameters to control pagination
params = {
    'per_page': 50  # Adjust this number as needed, up to the maximum allowed by the API
}

In [5]:
conversations_response = requests.get(f'{base_url}/conversations', headers=headers, params=params)
conversations = conversations_response.json()

# List to store all messages with sender names
sender_data = []

# Get sender names from starred conversations and set status to False
for conversation in conversations:
    if conversation["starred"]:
        sender_name = conversation["participants"][0]["name"]
        sender_data.append([sender_name])  # Appending [sender_name, False] to sender_data

print(sender_data)  # Print sender data for testing

[['Jacky Liu'], ['Subhan Baig']]


In [14]:
# Authenticate with Google Sheets API
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive", "https://www.googleapis.com/auth/spreadsheets"]
creds = ServiceAccountCredentials.from_json_keyfile_name('credentials.json', scope)
client = gspread.authorize(creds)

# Define the folder ID where you want to save the Google Sheets file
folder_id = os.getenv('GOOGLE_DRIVE_FOLDER_ID')
# Define file name for the Google Sheets file
file_name = 'Sender Data'

response = client.list_spreadsheet_files()
print(response)


# # Check if a file with the same name already exists
# file_exists = False
# for file in client.list_spreadsheet_files():
#     print(file['name'])
#     if file['name'] == file_name:
#         print("File already exists in the folder...")
#         file_exists = True
#         spreadsheet = client.open(file_name)
#         break

# # Get today's date and format it
# today_date = datetime.datetime.now().strftime("%Y-%m-%d")

# # If the file does not exist, create a new one and write sender data to it
# if not file_exists:
#     print('Creating a new file...')
#     spreadsheet = client.create(file_name, folder_id=folder_id)

#     # Rename the default worksheet ('Sheet1') with today's date
#     worksheet = spreadsheet.sheet1
#     # worksheet.update_title(today_date)
#     worksheet.append_row(['Sender Name'])  # Writing column headers

#     # Write sender data to the worksheet
#     for sender in sender_data:
#         worksheet.append_row(sender)  # Writing sender data

# else:
#     print('Creating a new worksheet...')
#     # Add a new worksheet with today's date as the title and write sender data to it
#     worksheet = spreadsheet.add_worksheet(title=today_date, rows="100", cols="20")  # Create a new worksheet
#     worksheet.append_row(['Sender Name'])  # Writing column headers

#     # Write sender data to the worksheet
#     for sender in sender_data:
#         worksheet.append_row(sender)  # Writing sender data

[{'id': '1OIzX5Tuep4Ro9tBttIpufQo7og201J45sO7k-V7er8M', 'name': 'Sender Data', 'createdTime': '2024-05-21T04:08:53.113Z', 'modifiedTime': '2024-05-21T04:08:58.638Z'}, {'id': '1qwtu2TORyO0OEYIpuNb9z_MOn6qge5o7LXnbzXoRMiQ', 'name': 'Sender Data', 'createdTime': '2024-05-21T04:07:32.852Z', 'modifiedTime': '2024-05-21T04:07:32.987Z'}, {'id': '1phWdD9wO3eOlha3gFb1J233Gk-zitV_ArjVvALXHgng', 'name': 'Sender Data', 'createdTime': '2024-05-21T04:06:27.105Z', 'modifiedTime': '2024-05-21T04:06:32.895Z'}, {'id': '1iNK2B8w4CyC04csALbq4sUyRIqPwQzELOcBwgDnOh4k', 'name': 'Sender Data', 'createdTime': '2024-05-21T04:06:01.899Z', 'modifiedTime': '2024-05-21T04:06:06.789Z'}, {'id': '1nUJcJ1s_Mi45upgcV8guznsDxdS77OnxRjx-_6OGmB0', 'name': 'Sender Data', 'createdTime': '2024-05-21T04:05:46.918Z', 'modifiedTime': '2024-05-21T04:05:50.100Z'}, {'id': '1-v85Y-64m8rSNcTNDmdEydaLdv5qqGVYWP2VojeLuPs', 'name': 'Sender Data', 'createdTime': '2024-05-21T04:04:18.180Z', 'modifiedTime': '2024-05-21T04:04:18.287Z'}, {'i